# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path = 'D:\\Part02\\'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""
# 選用cabin與Fare做群聚編碼
df['Cabin'] = df['Cabin'].fillna('None')
df_mean = df.groupby('Cabin')['Fare'].mean().reset_index()
df_median = df.groupby('Cabin')['Fare'].median().reset_index()
df_mode = df.groupby('Cabin')['Fare'].apply(lambda x:x.mode()[0]).reset_index()
df_max = df.groupby('Cabin')['Fare'].max().reset_index()
df_min = df.groupby('Cabin')['Fare'].min().reset_index()
df_count = df.groupby('Cabin')['Fare'].count().reset_index()
temp = pd.merge(df_mean,df_median,on='Cabin',how='left')
temp = pd.merge(temp,df_mode,on='Cabin',how='left')
temp = pd.merge(temp,df_max,on='Cabin',how='left')
temp = pd.merge(temp,df_min,on='Cabin',how='left')
temp = pd.merge(temp,df_count,on='Cabin',how='left')
temp.columns = ['Cabin','Fare_mean','Fare_median','Fare_mode','Fare_max','Fare_min','Fare_count']
temp.head()

,Cabin,Fare_mean,Fare_median,Fare_mode,Fare_max,Fare_min,Fare_count
0,A10,40.1250,40.1250,40.1250,40.1250,40.1250,1
1,A14,52.0000,52.0000,52.0000,52.0000,52.0000,1
2,A16,39.6000,39.6000,39.6000,39.6000,39.6000,1
3,A19,26.0000,26.0000,26.0000,26.0000,26.0000,1
4,A20,56.9292,56.9292,56.9292,56.9292,56.9292,1


In [4]:
df = pd.merge(df,temp,on='Cabin',how='left')
df = df.drop(['Cabin'],axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Fare_mean,Fare_median,Fare_mode,Fare_max,Fare_min,Fare_count
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,19.157325,10.5000,8.0500,512.3292,0.0000,687
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,71.283300,71.2833,71.2833,71.2833,71.2833,1
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,19.157325,10.5000,8.0500,512.3292,0.0000,687
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,53.100000,53.1000,53.1000,53.1000,53.1000,2
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,19.157325,10.5000,8.0500,512.3292,0.0000,687


In [5]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

11 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Fare_mean', 'Fare_median', 'Fare_mode', 'Fare_max', 'Fare_min', 'Fare_count']



,Pclass,Age,SibSp,Parch,Fare,Fare_mean,Fare_median,Fare_mode,Fare_max,Fare_min,Fare_count
0,3,22.0,1,0,7.2500,19.157325,10.5000,8.0500,512.3292,0.0000,687
1,1,38.0,1,0,71.2833,71.283300,71.2833,71.2833,71.2833,71.2833,1
2,3,26.0,0,0,7.9250,19.157325,10.5000,8.0500,512.3292,0.0000,687
3,1,35.0,1,0,53.1000,53.100000,53.1000,53.1000,53.1000,53.1000,2
4,3,35.0,0,0,8.0500,19.157325,10.5000,8.0500,512.3292,0.0000,687


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [6]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
df_original = df.drop(['Fare_mean','Fare_median','Fare_mode','Fare_max','Fare_min','Fare_count'],axis=1)
train_X = MMEncoder.fit_transform(df_original)
estimator = LogisticRegression()
print(cross_val_score(estimator,train_X,train_Y,cv=5).mean())

0.7038635542329971


In [7]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
train_X = MMEncoder.fit_transform(df)
estimator = LogisticRegression()
print(cross_val_score(estimator,train_X,train_Y,cv=5).mean())

0.6903740601683743


以選用cabin與Fare做群聚編碼的新特徵並沒有因此改善生存率預估

## 重新選用特徵來做群聚特徵

In [8]:
# 選用Embarked與Fare來做群聚特徵
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [9]:
df['Embarked'] = df['Embarked'].fillna('None')
df_mean = df.groupby('Embarked')['Fare'].mean().reset_index()
df_median = df.groupby('Embarked')['Fare'].median().reset_index()
df_mode = df.groupby('Embarked')['Fare'].apply(lambda x:x.mode()[0]).reset_index()
df_max = df.groupby('Embarked')['Fare'].max().reset_index()
df_min = df.groupby('Embarked')['Fare'].min().reset_index()
df_count = df.groupby('Embarked')['Fare'].count().reset_index()
temp = pd.merge(df_mean,df_median,on='Embarked',how='left')
temp = pd.merge(temp,df_mode,on='Embarked',how='left')
temp = pd.merge(temp,df_max,on='Embarked',how='left')
temp = pd.merge(temp,df_min,on='Embarked',how='left')
temp = pd.merge(temp,df_count,on='Embarked',how='left')
temp.columns = ['Embarked','Fare_mean','Fare_median','Fare_mode','Fare_max','Fare_min','Fare_count']
temp.head()

,Embarked,Fare_mean,Fare_median,Fare_mode,Fare_max,Fare_min,Fare_count
0,C,59.954144,29.70,7.2292,512.3292,4.0125,168
1,None,80.000000,80.00,80.0000,80.0000,80.0000,2
2,Q,13.276030,7.75,7.7500,90.0000,6.7500,77
3,S,27.079812,13.00,8.0500,263.0000,0.0000,644


In [10]:
df = pd.merge(df,temp,on='Embarked',how='left')
df = df.drop(['Embarked'],axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Fare_mean,Fare_median,Fare_mode,Fare_max,Fare_min,Fare_count
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,27.079812,13.0,8.0500,263.0000,0.0000,644
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,59.954144,29.7,7.2292,512.3292,4.0125,168
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,27.079812,13.0,8.0500,263.0000,0.0000,644
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,27.079812,13.0,8.0500,263.0000,0.0000,644
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,27.079812,13.0,8.0500,263.0000,0.0000,644


In [11]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

11 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Fare_mean', 'Fare_median', 'Fare_mode', 'Fare_max', 'Fare_min', 'Fare_count']



,Pclass,Age,SibSp,Parch,Fare,Fare_mean,Fare_median,Fare_mode,Fare_max,Fare_min,Fare_count
0,3,22.0,1,0,7.2500,27.079812,13.0,8.0500,263.0000,0.0000,644
1,1,38.0,1,0,71.2833,59.954144,29.7,7.2292,512.3292,4.0125,168
2,3,26.0,0,0,7.9250,27.079812,13.0,8.0500,263.0000,0.0000,644
3,1,35.0,1,0,53.1000,27.079812,13.0,8.0500,263.0000,0.0000,644
4,3,35.0,0,0,8.0500,27.079812,13.0,8.0500,263.0000,0.0000,644


In [12]:
# 原始特徵 + 邏輯斯迴歸

df_original = df.drop(['Fare_mean','Fare_median','Fare_mode','Fare_max','Fare_min','Fare_count'],axis=1)
train_X = MMEncoder.fit_transform(df_original)
estimator = LogisticRegression()
print(cross_val_score(estimator,train_X,train_Y,cv=5).mean())

0.7038635542329971


In [13]:
# 新特徵 + 邏輯斯迴歸

train_X = MMEncoder.fit_transform(df)
estimator = LogisticRegression()
print(cross_val_score(estimator,train_X,train_Y,cv=5).mean())

0.689256528950591


In [14]:
# 只加一些新特徵+邏輯斯迴歸

df_some_new = df.drop(['Fare_min','Fare_count'],axis=1)
train_X = MMEncoder.fit_transform(df_some_new)
estimator = LogisticRegression()
print(cross_val_score(estimator,train_X,train_Y,cv=5).mean())

0.7005180178502843


## 重新選用特徵來做群聚特徵

In [15]:
# 選用性別與年齡來做群聚特徵
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [16]:
df['Sex'] = df['Sex'].fillna('None')
df_mean = df.groupby('Sex')['Age'].mean().reset_index()
df_median = df.groupby('Sex')['Age'].median().reset_index()
df_mode = df.groupby('Sex')['Age'].apply(lambda x:x.mode()[0]).reset_index()
df_max = df.groupby('Sex')['Age'].max().reset_index()
df_min = df.groupby('Sex')['Age'].min().reset_index()
df_count = df.groupby('Sex')['Age'].count().reset_index()
temp = pd.merge(df_mean,df_median,on='Sex',how='left')
temp = pd.merge(temp,df_mode,on='Sex',how='left')
temp = pd.merge(temp,df_max,on='Sex',how='left')
temp = pd.merge(temp,df_min,on='Sex',how='left')
temp = pd.merge(temp,df_count,on='Sex',how='left')
temp.columns = ['Sex','Age_mean','Age_median','Age_mode','Age_max','Age_min','Age_count']
temp.head()

,Sex,Age_mean,Age_median,Age_mode,Age_max,Age_min,Age_count
0,female,27.915709,27.0,24.0,63.0,0.75,261
1,male,30.726645,29.0,19.0,80.0,0.42,453


In [17]:
df = pd.merge(df,temp,on='Sex',how='left')
df = df.drop(['Sex'],axis=1)
df.head()

,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_mean,Age_median,Age_mode,Age_max,Age_min,Age_count
0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S,30.726645,29.0,19.0,80.0,0.42,453
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C,27.915709,27.0,24.0,63.0,0.75,261
2,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,27.915709,27.0,24.0,63.0,0.75,261
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,S,27.915709,27.0,24.0,63.0,0.75,261
4,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,S,30.726645,29.0,19.0,80.0,0.42,453


In [18]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

11 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Age_mean', 'Age_median', 'Age_mode', 'Age_max', 'Age_min', 'Age_count']



,Pclass,Age,SibSp,Parch,Fare,Age_mean,Age_median,Age_mode,Age_max,Age_min,Age_count
0,3,22.0,1,0,7.2500,30.726645,29.0,19.0,80.0,0.42,453
1,1,38.0,1,0,71.2833,27.915709,27.0,24.0,63.0,0.75,261
2,3,26.0,0,0,7.9250,27.915709,27.0,24.0,63.0,0.75,261
3,1,35.0,1,0,53.1000,27.915709,27.0,24.0,63.0,0.75,261
4,3,35.0,0,0,8.0500,30.726645,29.0,19.0,80.0,0.42,453


In [20]:
# 原始特徵 + 邏輯斯迴歸

df_original = df.drop(['Age_mean','Age_median','Age_mode','Age_max','Age_min','Age_count'],axis=1)
train_X = MMEncoder.fit_transform(df_original)
estimator = LogisticRegression()
print(cross_val_score(estimator,train_X,train_Y,cv=5).mean())

0.7038635542329971


In [21]:
# 新特徵 + 邏輯斯迴歸

train_X = MMEncoder.fit_transform(df)
estimator = LogisticRegression()
print(cross_val_score(estimator,train_X,train_Y,cv=5).mean())

0.7822774202448624


以性別Sex與年紀Age所組合成的聚合特徵，確實能改善整個生存率預估